<a href="https://colab.research.google.com/github/jserrataylor/cursoAI/blob/main/Ejercicio_para_Kike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import joblib

# 1. Generar datos sintéticos
np.random.seed(42)
n_samples = 200
data = pd.DataFrame({
    'age': np.random.randint(18, 90, size=n_samples),
    'income': np.random.uniform(20000, 100000, size=n_samples),
    'num_visits': np.random.randint(0, 50, size=n_samples),
})

# Utilizamos una función logística para definir la probabilidad de compra
# Coeficientes arbitrarios para simulación
coef = np.array([0.03, 0.00002, 0.05])
intercept = -5
X = data.values
logit = X.dot(coef) + intercept
prob = 1 / (1 + np.exp(-logit))
data['purchased'] = (np.random.rand(n_samples) < prob).astype(int)

# 2. División entrenamiento / prueba
X = data[['age', 'income', 'num_visits']]
y = data['purchased']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Definición y entrenamiento de modelos
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    results[name] = acc
    print(f"{name}: Accuracy = {acc:.4f}")

# 4. Selección del mejor modelo
best_name = max(results, key=results.get)
best_model = models[best_name]
print(f"\nMejor modelo: {best_name} con accuracy = {results[best_name]:.4f}")

# 5. Guardar el mejor modelo
joblib.dump(best_model, 'best_model.pkl')
print("Modelo guardado como 'best_model.pkl'")


LogisticRegression: Accuracy = 0.7000
RandomForest: Accuracy = 0.6250
GradientBoosting: Accuracy = 0.6500

Mejor modelo: LogisticRegression con accuracy = 0.7000
Modelo guardado como 'best_model.pkl'


In [ ]:
import streamlit as st
import joblib
import numpy as np

# Título y descripción
st.title("Predicción de Compra de Cliente")
st.write("""
Este servicio despliega un modelo de clasificación entrenado sobre datos sintéticos
y predice la probabilidad de que un cliente compre un producto en función de:
- Edad
- Ingresos anuales
- Número de visitas al sitio
""")

# Función para cargar y cachear el modelo
@st.cache(allow_output_mutation=True)
def load_model():
    return joblib.load("best_model.pkl")

model = load_model()

# Panel lateral para los parámetros de entrada
st.sidebar.header("Parámetros de entrada")
age = st.sidebar.slider("Edad", min_value=18, max_value=90, value=30, step=1)
income = st.sidebar.number_input("Ingresos anuales (USD)", min_value=0.0, value=50000.0, step=1000.0)
visits = st.sidebar.slider("Número de visitas al sitio", min_value=0, max_value=50, value=5, step=1)

# Preparar los datos de entrada y hacer la predicción
X_new = np.array([[age, income, visits]])
prob = model.predict_proba(X_new)[0, 1]
pred = model.predict(X_new)[0]

# Mostrar resultados
st.subheader("Resultados de la predicción")
st.markdown(f"- **Probabilidad de compra:** {prob:.2%}")
st.markdown(f"- **Predicción:** {'🛒 Comprar' if pred == 1 else '🚫 No comprar'}")
